In [1]:
import sys
import os
from os.path import join
import torch
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from ipywidgets import interact_manual,interact, fixed
sys.path.append('..')

Functions

In [2]:
def extract_tensors(path, tensor_name):
    '''extracts tensors from path (as created when any uncertainty test is ran'''
    # get only the correct folders
    folders = []
    for sub_path in os.listdir(path):
        if sub_path.startswith('image'):
            folders.append( sub_path)
    
    # extract tensors
    all_images = {}
    for im_folder in folders:
        im_path = join(path, im_folder)
        # save the tensors in their own dict according to name before
        for tensor in os.listdir(im_path):
            if tensor == tensor_name:
                tensor_path = join(im_path, tensor)
                all_images[int(im_folder.split('_')[-1])] = torch.load(tensor_path, map_location = 'cpu')
    return all_images


In [3]:
def display_tensor(tensor_dict:dict, im_dict:dict, target_dict:dict, im_num: int = 0 , vmin:float = 0, vmax: float = 1):
    fig, ax = plt.subplots(1,3, figsize = (18, 12))
    im1 = ax[0].imshow(im_dict[im_num], cmap = 'gray',)
    im2 = ax[1].imshow(tensor_dict[im_num][0][0].numpy(), cmap = 'gray', vmin = vmin, vmax = vmax)
    im3 = ax[2].imshow(target_dict[im_num], cmap = 'gray', )
    ax[0].set_title('Orig')
    ax[1].set_title('Expected')
    ax[2].set_title('GT')
    fig.show()
    print('Min: ' ,torch.min(tensor_dict[im_num][0][0]).item())
    print('Max: ' ,torch.max(tensor_dict[im_num][0][0]).item())

In [4]:
def compare_tensor(tensor_dict1:dict, tensor_dict2:dict, im_dict:dict, target_dict:dict, im_num: int = 0 , vmin:float = 0, vmax: float = 1):
    fig, ax = plt.subplots(1,4, figsize = (30, 28))
    im1 = ax[0].imshow(im_dict[im_num], cmap = 'gray',)
    im2 = ax[1].imshow(tensor_dict1[im_num][0][0].numpy(), cmap = 'gray', vmin = vmin, vmax = vmax)
    im3 = ax[2].imshow(tensor_dict2[im_num][0][0].numpy(), cmap = 'gray', vmin = vmin, vmax = vmax)
    im4 = ax[3].imshow(target_dict[im_num], cmap = 'gray', )
    ax[0].set_title('Orig')
    ax[1].set_title('Independent')
    ax[2].set_title('Dependent')
    ax[3].set_title('GT')
    fig.show()
    print(f'Min: I:{torch.min(tensor_dict1[im_num][0][0]).item():7.5f} D: {torch.min(tensor_dict2[im_num][0][0]).item():7.5f}', )
    print(f'Min: I:{torch.max(tensor_dict1[im_num][0][0]).item():7.5f} D: {torch.max(tensor_dict2[im_num][0][0]).item():7.5f}', )

In [5]:
def pseudo_anim(tensor_dict: dict, val_num: int = 0 , im_num: int = 0 ):
    ''' creates an animated gif using images
    
    output:
    creates a gif'''

    fig, ax = plt.subplots(1,1, figsize = (8, 8))
    ax.imshow(tensor_dict[val_num][im_num][0][0].numpy(), cmap = 'gray', vmin = 0, vmax = 1)
    fig.show()
    

Root Directory - change if different (make sure corresponding internal files are also changed)

In [6]:
root = '../../results/base_model'

print(os.listdir(root))

['model_info', 'lightning_logs', 'test_statistics', 'dropblock_uncertainty_i', 'rotational_uncertainty', 'test_statistics_32_32', 'test_statistics_128_128', 'test_statistics_64_64', 'test_statistics_256_256', 'dropblock_uncertainty_d']


Validation Images

In [7]:
val_root_images = '../../augmented_data/val/images'
orig = {}
# retrieves validation images as grayscale
for im in os.listdir(val_root_images):
    orig[int(im.split('_')[0])] = np.array(Image.open(join(val_root_images, im)).convert('L'))

val_root_targets = '../../augmented_data/val/targets'
targets = {}
for im in os.listdir(val_root_targets):
    targets[int(im.split('_')[0])] = np.array(Image.open(join(val_root_targets, im)).convert('L'))


# Monte Carlo Dropblock - Dependent Channels

In [12]:
DB_d_path = join(root, 'dropblock_uncertainty_d/tensors')

DB_d_mean_tensors = extract_tensors(DB_d_path, 'mean.pt')
DB_d_std_tensors = extract_tensors(DB_d_path, 'std.pt')
DB_d_extra_tensors = extract_tensors(DB_d_path, 'tensors.pt')

Means

In [13]:
x = interact(display_tensor,
        tensor_dict=fixed(DB_d_mean_tensors),
        im_dict = fixed(orig),
        target_dict = fixed(targets),
        im_num = (0,len(DB_d_mean_tensors) -1),
        vmin = (0.0,1.0, .01 ),
        vmax = (0.0, 1.0, .01))

interactive(children=(IntSlider(value=0, description='im_num', max=5), FloatSlider(value=0.0, description='vmi…

STD

In [14]:
x = interact(display_tensor,
        tensor_dict=fixed(DB_d_std_tensors),
        im_dict = fixed(orig),
        target_dict = fixed(targets),
        im_num = (0,len(DB_d_std_tensors) -1),
        vmin = (0.0,1.0, .01 ),
        vmax = (0.0, 1.0, .01), )

interactive(children=(IntSlider(value=0, description='im_num', max=5), FloatSlider(value=0.0, description='vmi…

View 25 Examples

In [ ]:
x = interact(pseudo_anim, 
                tensor_dict = fixed(DB_d_extra_tensors),
                val_num =  (0,len(DB_d_extra_tensors) -1),
                im_num = (0, len(DB_d_extra_tensors[0]) -1))

# Monte Carlo Dropblock = Independent

In [18]:
DB_i_path = join(root, 'dropblock_uncertainty_i/tensors')

DB_i_mean_tensors = extract_tensors(DB_i_path, 'mean.pt')
DB_i_std_tensors = extract_tensors(DB_i_path, 'std.pt')
DB_i_extra_tensors = extract_tensors(DB_i_path, 'tensors.pt')

Mean

In [19]:
x = interact(display_tensor,
        tensor_dict=fixed(DB_i_mean_tensors),
        im_dict = fixed(orig),
        target_dict = fixed(targets),
        im_num = (0,len(DB_i_mean_tensors) -1),
        vmin = (0.0,1.0, .01 ),
        vmax = (0.0, 1.0, .01), )

interactive(children=(IntSlider(value=0, description='im_num', max=5), FloatSlider(value=0.0, description='vmi…

STD

In [20]:
x = interact(display_tensor,
        tensor_dict=fixed(DB_i_std_tensors),
        im_dict = fixed(orig),
        target_dict = fixed(targets),
        im_num = (0,len(DB_i_std_tensors) -1),
        vmin = (0.0,1.0, .01 ),
        vmax = (0.0, 1.0, .01), )

interactive(children=(IntSlider(value=0, description='im_num', max=5), FloatSlider(value=0.0, description='vmi…

View 25 Tensors

In [21]:
interact(pseudo_anim, 
                tensor_dict = fixed(DB_i_extra_tensors),
                val_num =  (0,len(DB_i_extra_tensors) -1),
                im_num = (0, len(DB_i_extra_tensors[0]) -1))

interactive(children=(IntSlider(value=0, description='val_num', max=5), IntSlider(value=0, description='im_num…

<function __main__.pseudo_anim(tensor_dict: dict, val_num: int = 0, im_num: int = 0)>

# Independent vs Dependent

Mean

In [22]:
x = interact(compare_tensor,
        tensor_dict1=fixed(DB_i_mean_tensors),
        tensor_dict2=fixed(DB_d_mean_tensors),
        im_dict = fixed(orig),
        target_dict = fixed(targets),
        im_num = (0,len(DB_i_mean_tensors) -1),
        vmin = (0.0,1.0, .01 ),
        vmax = (0.0, 1.0, .01), )

interactive(children=(IntSlider(value=0, description='im_num', max=5), FloatSlider(value=0.0, description='vmi…

STD

In [23]:
x = interact(compare_tensor,
        tensor_dict1=fixed(DB_i_std_tensors),
        tensor_dict2=fixed(DB_d_std_tensors),
        im_dict = fixed(orig),
        target_dict = fixed(targets),
        im_num = (0,len(DB_i_std_tensors) -1),
        vmin = (0.0,1.0, .01 ),
        vmax = (0.0, 1.0, .01), )

interactive(children=(IntSlider(value=0, description='im_num', max=5), FloatSlider(value=0.0, description='vmi…

# Rotational Uncertainty

In [24]:
rotation_path = join(root, 'rotational_uncertainty')

rotation_mean_tensors = extract_tensors(rotation_path, 'mean.pt')
rotation_std_tensors = extract_tensors(rotation_path, 'std.pt')
rotation_extra_tensors = extract_tensors(rotation_path, 'tensors.pt')

In [25]:
rotation_mean_tensors[0].shape

torch.Size([1, 1, 584, 565])

Mean

In [26]:
x = interact(display_tensor,
        tensor_dict=fixed(rotation_mean_tensors),
        im_dict = fixed(orig),
        target_dict = fixed(targets),
        im_num = (0,len(rotation_mean_tensors) -1),
        vmin = (0.0,1.0, .01 ),
        vmax = (0.0, 1.0, .01), )

interactive(children=(IntSlider(value=0, description='im_num', max=5), FloatSlider(value=0.0, description='vmi…

STD

In [27]:
x = interact(display_tensor,
        tensor_dict=fixed(rotation_std_tensors),
        im_dict = fixed(orig),
        target_dict = fixed(targets),
        im_num = (0,len(rotation_std_tensors) -1),
        vmin = (0.0,1.0, .01 ),
        vmax = (0.0, 1.0, .01), )

interactive(children=(IntSlider(value=0, description='im_num', max=5), FloatSlider(value=0.0, description='vmi…

View 25 Images

In [28]:
x = interact(pseudo_anim, 
                tensor_dict = fixed(rotation_extra_tensors),
                val_num =  (0,len(rotation_extra_tensors) -1),
                im_num = (0, len(rotation_extra_tensors[0]) -1))

interactive(children=(IntSlider(value=0, description='val_num', max=5), IntSlider(value=0, description='im_num…